In [10]:
import psycopg2
import requests
import pandas as pd
import io
import urllib3

First we need to establish the database connection. If you haven't already, get it running using docker compose

In [11]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432",
)

cur = conn.cursor()

Now to make any requests to the database for data we can use cur.execute()

In [13]:
cur.execute("SELECT * FROM machine_links INNER JOIN features ON machine_links.ccn = features.ccn WHERE cardinality(csv_headers) is NULL") # query to get joined data
res = cur.fetchall()
df = pd.DataFrame(res)

df = df.iloc[:, 0:9] # remove the double ccn column
df.columns = ["ccn", "state_or_region", "reporting_entity_name_common", "machine_readable_url", "csv_headers", "meets_standard", "bed_count", "zip_code", "medicare_medicaid_eligible"]

df.set_index("ccn", inplace=True) # index by ccn
df

,state_or_region,reporting_entity_name_common,machine_readable_url,csv_headers,meets_standard,bed_count,zip_code,medicare_medicaid_eligible
ccn,,,,,,,,
010001,AL,Southeast Health Medical Center,https://www.southeasthealth.org/wp-content/upl...,None,False,420,36301,True
010005,AL,Marshall Medical Centers South Campus,https://pricetransparency.blob.core.windows.ne...,None,True,240,35957,True
010006,AL,North Alabama Medical Center,https://namccares.com/sites/namc/assets/upload...,None,False,338,35630,True
010011,AL,Saint Vincent's East,https://healthcare.ascension.org/-/media/proje...,None,False,362,35235,True
010012,AL,Dekalb Regional Medical Center,https://dekalbregional.com/wp-content/uploads/...,None,True,134,35968,True
...,...,...,...,...,...,...,...,...
673074,TX,Everest Rehabilitation Hospital Temple,https://www.everestrehab.com/Everest_Rehabilit...,None,True,36,76504,True
673076,TX,Clearsky Rehabilitation Hospital Of Flower Mound,https://clearskyhealth.com/flowermound/wp-cont...,None,False,41,75022,True
673078,TX,"Shannon Rehabilitation Hospital, An Affiliate ...",https://encompasshealth.com/-/media/healthsout...,None,True,40,76901,True


To add some more insight to the provided url, we can parse the file to see which headers are provided by each hospital

In [14]:
def get_headers(f):
    # f: csv file object
    # returns list of csv headers
    f.readline()
    d_reader = csv.DictReader(f)
    headers = d_reader.fieldnames

    return headers


def parse_mrf(url):
    # url: link to machine readable file
    response = requests.get(url)
    print(response.text)
    return get_headers(response)

In [15]:
for row in res:
    session = requests.Session()
    retry = urllib3.util.retry.Retry(connect=3, backoff_factor=.5, raise_on_status=False)
    adapter = requests.adapters.HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    try:
        r = session.get(row[3])
    except:
        continue
    print(r.status_code)
    print(row[3])
    ccn = row[0]
    if str(r.status_code)[0] == str(4) or str(r.status_code)[0] == str(5) or r.headers['Content-Type'] in ["application/pdf", "application/zip", "application/json"]:
        cur.execute(
            "UPDATE machine_links SET meets_standard=false WHERE ccn=%s", (str(ccn),)
        )
        cur.execute(
            "UPDATE machine_links SET csv_headers=array_append(csv_headers, %s) WHERE ccn=%s", (str(r.status_code), str(ccn))
        )
        conn.commit()
        continue
    if row[3][-4:] == "xlsx":
        headers = list(pd.read_excel(r.content, engine='openpyxl').columns)
    elif row[3][-3:] == "xls":
        headers = list(pd.read_excel(r.content, engine="xlrd").columns)
    elif row[3][-3:] == "csv":
        if row[3] in ["https://app.peacehealth.org/FileRepository/Financial/StandardCharges/KEMC/92-0016490_Ketchikan_Medical_Center_StandardCharges.csv", "https://apps.scripps.org/pricetransparency/951684089_ScrippsMercyHospitalSanDiego_standardcharges.csv"]:
            headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore")), delimiter="|").columns)
        else:
            headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)
    elif row[3][-3:] == "zip":
        continue
    cur.execute(
        "UPDATE machine_links SET csv_headers=%s WHERE ccn=%s", (headers, ccn, )
    )
    conn.commit()

200
https://www.southeasthealth.org/wp-content/uploads/Southeast-Health-Standard-Charges-2022.xlsx
200
https://pricetransparency.blob.core.windows.net/mmc/83-1651180_marshallmedicalcenters_standardcharges.csv


/var/folders/29/818nv1qj7z1c_zb46ggt489m0000gn/T/ipykernel_61477/4094791315.py:31: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)


200
https://namccares.com/sites/namc/assets/uploads/CDM%20-%20North%20Alabama%20MC%2023200.xlsx
406
https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/al/630578923_ascension-saint-vincents-east_standardcharges.xlsx


IndexError: tuple index out of range